<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello!

If you're seeing this on GitHub, you can open the document on Google by clicking the "Open in Collab" button above.  Once this document is open in Google, you can see the interactive content by selecting `Runtime` on the menubar and then `Run All.`


# Set up environment


In [1]:
import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler, scale

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap
from MightyLogic.TurfWar.Tiles.TileList import TileList
#Values.GOLD

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-5s4uatdq/mightylogic_aa719782ae0a47819f4e7a0822e1218e
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-5s4uatdq/mightylogic_aa719782ae0a47819f4e7a0822e1218e
     |████████████████████████████████| 280 kB 5.1 MB/s 
     |████████████████████████████████| 40 kB 4.4 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
  Created wheel for MightyLogic: filename=MightyLogic-0.2-py3-none-any.whl size=23523 sha256=f35e5de8cd929800890892e42b9a824da4066aa3f5b93990a979fc6fc038bad6
  Stored in directory: /tmp/pip-ephem-wheel-cache-w_yniww4/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic
  Attempting uninstall: zipp
    Found existing installation: zipp 3.5.0
    Uninstalling zipp-3.5.0:
      Successfully unins

# Turf War Demo

In [2]:

#
# A Turf War Map
#

map2 =TurfWarMap()
map2.setGuild("Nightterror")
map2.setStartDate(year=2021, month=7, day=17)


# Use convenience functions to build value array
map2.addTile('A',1, mountains(3000,2000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000, 2000))

map2.addTile('B',1, factory(1650,500,5000))
map2.addTile('B',2, mountains(5000,5000))
map2.addTile('B',3, lighthouse(3500,300,7000))
map2.addTile('B',4, plains(5000,5000))
map2.addTile('B',5, mountains(5000,5000))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(5000,5000))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, hells_gate(600,800,8500))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, witch_hut(3500,600, 7000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, altar(600,800,8500))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, lighthouse(2000, 400, 5000))

map2.addTile('E',1, plains(3000, 2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, forest(3000,0,2000))
map2.addTile('E',4, plains(3000, 2000))
map2.addTile('E',5, mountains(3000,2000))
map2.addTile('E',6, forest(3000,0,2000))


# Payout array
image = map2.getValues()
res = map2.stagingScores()
#pprint(image)

In [3]:
#
# An approach that shows well-labeled heatmaps.
#

def get_heatmap(arr, title, round=True):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip
  if round:
    z = z.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  if round:
    z_text = flip.round( decimals=-1)
  else:
    z_text = flip.round( decimals=3)
  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  return fig #fig.show()



In [4]:
#
# The payout array can be accessed as a Pandas dataframe.
#
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,30003.0,30003.0,30003.0,15001.6,30003.0,30003.0
B,82582.5,75005.0,109535.0,75005.0,75005.0,136500.0
C,30003.0,75005.0,75005.0,75005.0,139560.0,30003.0
D,30003.0,114035.0,75005.0,139560.0,75005.0,81020.0
E,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0


In [5]:
#
# The staging array can be accessed as a Pandas dataframe.
#
staging =map2.getStrategicDataFrame()
staging

,1,2,3,4,5,6
A,112585.5,135011.0,154539.6,135011.0,120009.6,166503.0
B,0.0,297125.5,0.0,274546.6,381068.0,0.0
C,187590.5,294048.0,334550.0,429130.0,0.0,357080.0
D,174041.0,0.0,358603.0,0.0,390143.0,0.0
E,60006.0,174041.0,135011.0,199566.0,135011.0,111023.0


In [6]:
#
# Heatmaps show the best tiles for payout round and non-payout rounds.
#


fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
fig2 = get_heatmap(res,"Strategic Values for Non-Payout Rounds")

fig1.show()
fig2.show()

In [7]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('B',6) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                 4,500 gold, 250 contribution
4-6                 3,600 gold, 200 contribution
7-11                2,700 gold, 150 contribution
12-25               1,800 gold, 100 contribution
26-44               900 gold, 50 contribution
500 guild influence


In [8]:
#
# List of buildings easy for players to read.
#
map2.printBuildingList()

Building          Points      For 1st Place                      Guild Gets
C-5: Hell's Gate  139,560      42 epic souls, 595 contribution,  800 influence.
D-4: Altar        139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
D-2: Witch Hut    114,035     245 rare souls, 490 contribution,  600 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
D-6: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.


In [9]:
#
# Text report that highlights building returns 
#

map2.printReport()

Turf War Map Report
Guild: Nightterror
For the Week of: 17 July 2021
Total Map Value: 1,837,876 common souls

Building          Points      For 1st Place                      Guild Gets
C-5: Hell's Gate  139,560      42 epic souls, 595 contribution,  800 influence.
D-4: Altar        139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
D-2: Witch Hut    114,035     245 rare souls, 490 contribution,  600 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
D-6: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.


In [10]:
#
# Evaluate neighbors of a given tile
#
map2.printNeighborList('B',2)

Neighbors of tile B:2
Building          Points      For 1st Place                      Guild Gets
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
A-2: Forest        30,003   210 common souls, 140 contribution,    0 influence.
C-2: Plains        75,005   350 common souls, 350 contribution,    0 influence.


# Strategy & Payout Arrays

In [11]:
#
# High values are desirable payout locations, low are less desirable
#

#pd.DataFrame(map2.getScaledPayout())

In [12]:
#
# High values are desirable staging locations, low are less desirable
#

#pd.DataFrame(map2.getScaledStaging())

In [13]:
#
# Lowest values are least desirable staging locations, higher are better staging locations, highest are desirable payout locations
#
adjusted_move = map2.getAdjustedMoves()
pd.DataFrame(adjusted_move)

,0,1,2,3,4,5
0,0.066253,0.066253,0.066253,0.000000,0.066253,0.078954
1,1.484856,1.280413,2.212038,1.280413,1.280413,2.939557
2,0.131822,1.280413,1.280413,1.280413,3.022117,0.556750
3,0.097852,2.333448,1.280413,3.022117,1.280413,1.442699
4,0.066253,0.097852,0.066253,0.161846,0.066253,0.066253


In [14]:
moves = adjusted_move
get_heatmap(moves,"Movement Array", round=False).show()

In [15]:
#
# 3D Presentation of the movement surface
#
flip = np.flip( moves, axis=1) 
fig = go.Figure(data=[go.Surface(z=flip)])
fig.update_layout(title='Movement Array in 3D', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [16]:
retx, rety = map2.getRecommendedMovesArrays()

In [17]:
def getTileTrace():

  # Generate tile name array
  cnames = []
  rtl = ['E','D','C','B','A']
  ctli = [1,2,3,4,5,6]
  for c in ctli:
    for r in rtl:
      cnames.append(f"{r}{c}")
      
  return go.Scatter(x=[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5], y=[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                          mode='markers+text',
                          textposition="top right",
                          text = cnames,
                          textfont=dict(
                            family="sans serif",
                            size=18,
                            color="red"
                          ),
                          marker_size=14,
                          name='Tiles')
  
def getContourTrace(data):
  z = np.flip( data, axis=0)
  return(go.Contour(
        z=z,
        x=[0,1,2,3,4,5], # horizontal axis
        y=[0,1,2,3,4],   # vertical axis
        colorscale="delta"
        #showlegend=False
        #color_continuous_scale=px.colors.sequential.Viridis
    ))

def squaresTrace():
  pass
  #fig.add_shape(type="rect",
  #  xref="x", yref="y",
  #  x0=-0.5, y0=-0.5,
  #  x1=0.5, y1=0.5,
  #  line=dict(
  #      color="RoyalBlue",
  #      width=3,
  #  ),
  #  #fillcolor="LightSkyBlue",
  # #)

In [18]:
#
# Show staging values as a contour plot
#
fig = go.Figure()
fig.add_trace( getContourTrace(map2.getStrategicDataFrame().to_numpy()) )
fig.add_trace( getTileTrace())
fig.update_layout(title='Strategic Value of Staging Tiles')
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(dict(paper_bgcolor='White',plot_bgcolor='White', showlegend=False))
fig.show()

In [19]:
  #fig = ff.create_streamline(x, y, u,v, arrow_scale=.1, name='streamline')
  x,y = np.meshgrid(np.arange(0, 6, 1), np.arange(0, 5, 1))
  fig = ff.create_quiver(x,y,                       #positions 
                         retx, rety,                #vectors
                         scale=1,                   #vector scale
                         name="Recommended Move",   #legend title
                         line=dict(width=2,
                                color='red'),
                        )

  # add dots at integers
  fig.add_trace( getTileTrace())

  # add contour plot   
  z = payout.to_numpy()
  fig.add_trace( getContourTrace(z) )
  
  # cosmetics
  #  - axes are confusing & extraneous
  fig.update_xaxes(visible=False)
  fig.update_yaxes(visible=False)

  #  - KISS   
  showLegend = False #dots & arrows  # 3 legends are colliding
  fig.update_layout(dict(paper_bgcolor='White',plot_bgcolor='White', showlegend=showLegend))

  #fig.update_layout dict(show_legend=False))
  #fig.update_layout(coloraxis_showscale=False)

  # move colorbar
  #fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
  #                                        ticks="outside",
  #                                        ticksuffix=" bills"))

  # - graph title
  fig.update_layout(title='Relative Payout and Recommended Moves')
  fig.show()

# Multi-Tile Targets

In [20]:
import re
s = 'a1a5c6e3'
res = re.findall('(\d+|[A-Za-z]+)', s)
print(res)

def parse_tile(inp):
  pass

['a', '1', 'a', '5', 'c', '6', 'e', '3']


In [21]:
image = map2.getValues()


#pprint(image)
filter_kernel =  [[0, 0, 0],
                  [0, 1, 1],
                  [0, 1, 0]]

def getComboValues(filter_kernel):
  filter_kernel = np.flipud(np.fliplr(filter_kernel))
  res = convolve2d(image,
                 filter_kernel,
                 mode='same',
                 boundary='fill',
                 fillvalue=0)
  return res

#pprint(res)
#fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
#fig1.show()

filter_kernel0 =  [[0, 0, 0],
                  [0, 1, 1],
                  [0, 1, 0]]
drcombos = getComboValues(filter_kernel)

filter_kernel1 =  [[0, 1, 0],
                  [0, 1, 1],
                  [0, 0, 0]]
urcombos = getComboValues(filter_kernel)

filter_kernel2 =  [[0, 0, 0],
                  [1, 1, 0],
                  [0, 1, 0]]
dlcombos = getComboValues(filter_kernel)

filter_kernel3 =  [[0, 1, 0],
                  [1, 1, 0],
                  [0, 0, 0]]
ulcombos = getComboValues(filter_kernel)

filter_kernel4 =  [[0, 0, 0],
                  [1, 1, 1],
                  [0, 0, 0]]
hcombos = getComboValues(filter_kernel)

filter_kernel5 =  [[0, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0]]
vcombos = getComboValues(filter_kernel)

filter_kernels = [filter_kernel0,filter_kernel1,filter_kernel2,filter_kernel3,filter_kernel4,filter_kernel5]
#get_heatmap(drcombos,"down-right combos  ", round=False)
get_heatmap(urcombos,"up-right combos  ", round=False)
get_heatmap(hcombos,"horiz combos  ", round=False)
get_heatmap(vcombos,"vert combos  ", round=False)

#drlcombos.show()

In [31]:

# given row in [0-4] and col in [0-5], return array of valid tile locations
# matching the given convolution matrix

def getFilterTileLocations(r, c, filter_kernel):
  row_dict = {0:'A', 1:'B', 2: 'C', 3:'D', 4: 'E'}
  row=row_dict[r]
  j=c+1

  uprow = map2.upRow(row)
  leftcol = map2.leftCol(j)
  rightcol = map2.rightCol(j)
  dnrow = map2.downRow(row)

  ret = []
  print(uprow)
  # above row
  if uprow is not None:
    if leftcol is not None:
      if filter_kernel[0][0]==1:
        #print(uprow + str(leftcol))
        ret.append([uprow, leftcol])
    if filter_kernel[0][1]==1:
      #print(uprow + str(j))
      ret.append([uprow, j])
    if rightcol is not None:
      if filter_kernel[0][2]==1:
        #print(uprow + str(rightcol))
        ret.append([uprow, rightcol])

  # this row
  if leftcol is not None:
    if filter_kernel[1][0]==1:
      #print(row + str(leftcol))
      ret.append([row, leftcol])
  if filter_kernel[1][1]==1:
    #print(row + str(j))
    ret.append([row, j])
  if rightcol is not None:
    if filter_kernel[1][2]==1:
      #print(row + str(rightcol))
      ret.append([row, rightcol])

  # below row
  if dnrow is not None:
    if leftcol is not None:
      if filter_kernel[2][0]==1:
        #print(dnrow + str(leftcol))
        ret.append([dnrow, leftcol])
    if filter_kernel[2][1]==1:
      #print(dnrow + str(j))
      ret.append([dnrow, j])
    if rightcol is not None:
      if filter_kernel[2][2]==1:
        #print(dnrow + str(rightcol))
        ret.append([dnrow, rightcol])
  return ret

ret = getFilterTileLocations(0, 0, filter_kernel)
pprint(ret)
tarr = map2.getTiles(ret)
tlist = TileList(map2,ret)
#tlist.share(0.07)

None
[['A', 1], ['A', 2], ['B', 1]]


In [32]:
allCombos = []
for filter in filter_kernels:
  for r in [0,1,2,3,4]:
    for c in [0,1,2,3,4,5]:
      allCombos.append(TileList(map2, getFilterTileLocations(r, c, filter_kernel)))

allCombos = set(allCombos)

None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D


In [24]:
allCombos = list(allCombos)
allCombos.sort(key=lambda x: x.value, reverse=True)
for combo in allCombos:
  val = combo.getValue()
  print(combo.getTileLine()) #str(val) + " - " + combo.getName())

C:4, C:5, D:4:  354,125  350 common souls, 84 epic souls, 1,540 contribution, 1600 influence.
B:5, B:6, C:5:  351,065  350 common souls, 42 epic souls, 6,300 gold, 1,295 contribution, 1300 influence.
C:2, C:3, D:2:  264,045  700 common souls, 245 rare souls, 1,190 contribution,  600 influence.
B:2, B:3, C:2:  259,545  700 common souls, 245 rare souls, 1,190 contribution,  300 influence.
B:3, B:4, C:3:  259,545  700 common souls, 245 rare souls, 1,190 contribution,  300 influence.
C:5, C:6, D:5:  244,568  560 common souls, 42 epic souls, 1,085 contribution,  800 influence.
D:3, D:4, E:3:  244,568  560 common souls, 42 epic souls, 1,085 contribution,  800 influence.
D:4, D:5, E:4:  244,568  560 common souls, 42 epic souls, 1,085 contribution,  800 influence.
B:4, B:5, C:4:  225,015  1,050 common souls, 1,050 contribution,    0 influence.
C:3, C:4, D:3:  225,015  1,050 common souls, 1,050 contribution,    0 influence.
D:2, D:3, E:2:  219,043  560 common souls, 245 rare souls, 980 contribu

In [30]:

topTen = allCombos[:10]
print("Top 3-Tile Combos on Map")
print("Tiles           Points   Reward for first place")
print("--------------------------------------------------------------------------------------------------------")
for combo in topTen:
  print(combo.getTileLine())

Top 3-Tile Combos on Map
Tiles           Points   Reward for first place
--------------------------------------------------------------------------------------------------------
C:4, C:5, D:4:  354,125  350 common souls, 84 epic souls, 1,540 contribution, 1600 influence.
B:5, B:6, C:5:  351,065  350 common souls, 42 epic souls, 6,300 gold, 1,295 contribution, 1300 influence.
C:2, C:3, D:2:  264,045  700 common souls, 245 rare souls, 1,190 contribution,  600 influence.
B:2, B:3, C:2:  259,545  700 common souls, 245 rare souls, 1,190 contribution,  300 influence.
B:3, B:4, C:3:  259,545  700 common souls, 245 rare souls, 1,190 contribution,  300 influence.
C:5, C:6, D:5:  244,568  560 common souls, 42 epic souls, 1,085 contribution,  800 influence.
D:3, D:4, E:3:  244,568  560 common souls, 42 epic souls, 1,085 contribution,  800 influence.
D:4, D:5, E:4:  244,568  560 common souls, 42 epic souls, 1,085 contribution,  800 influence.
B:4, B:5, C:4:  225,015  1,050 common souls, 1,050 cont

In [33]:
best = TileList(map2, [['C', 4], ['C', 5], ['D', 4]])
print(best.payouts())


Payouts for Forest at C:4, Hell's Gate at C:5, Altar at D:4
Place               Reward
First               350 common souls, 84 epic souls, 1,540 contribution
2nd                 300 common souls, 72 epic souls, 1,320 contribution
3rd                 250 common souls, 60 epic souls, 1,100 contribution
4-6                 200 common souls, 48 epic souls, 880 contribution
7-11                150 common souls, 36 epic souls, 660 contribution
12-25               100 common souls, 24 epic souls, 440 contribution
26-44               50 common souls, 12 epic souls, 220 contribution



# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb
1. Add payout report for multiple tiles

